### РОССИЙСКИЙ УНИВЕРСИТЕТ ДРУЖБЫ НАРОДОВ
#### Факультет физико-математических и естественных наук  
#### Кафедра информационных технологий 

# ОТЧЕТ ПО ЛАБОРАТОРНОЙ РАБОТЕ № 2
# Поиск ассоциативных правил
### Дисциплина: Интеллектуальный анализ данных


##### Студент:  Пономарева Л. М.
##### Группа:   НПИбд-02-19

### Москва 2022

### Вариант №10

Алгоритм: Apriori 

День недели (поле order_dow таблицы orders): “6” 

Код департамента (поле department_id таблицы products): “2” 

Показатель оценки ассоциативных правил: лифт (lift) 

### Задание 1
При помощи модуля sqlite3 откройте базу данных Instacart в файле instacart.db.

In [1]:
import numpy as np
import pandas as pd
import sqlite3

conn = sqlite3.connect('instacart.db')

### Задание 2
Загрузите таблицы `departments` и `products` в датафреймы Pandas. При помощи запроса SELECT извлеките из таблицы `order_products__train` записи, соответствующие указанным в индивидуальном задании дню недели (поле `order_dow` таблицы `orders`) и коду департамента (поле `department_id` таблицы `products`) и загрузите в датафрейм Pandas. Определите количество строк в полученном датафрейме и определите количество товаров (столбец `product_id`) в транзакциях датафрейма. 


Загрузим таблицы `departments` и `products` в датафреймы Pandas.

In [2]:
df_dept = pd.read_sql_query("SELECT * FROM departments", conn)
df_dept

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


In [10]:
df_prod = pd.read_sql_query("SELECT * FROM products", conn)
df_prod

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13
...,...,...,...,...
49683,49684,"Vodka, Triple Distilled, Twist of Vanilla",124,5
49684,49685,En Croute Roast Hazelnut Cranberry,42,1
49685,49686,Artisan Baguette,112,3
49686,49687,Smartblend Healthy Metabolism Dry Cat Food,41,8


Извлечем из таблицы `order_products__train` записи, соответствующие дню недели = 6 (поле `order_dow` таблицы `orders`) и коду департамента = 2 (поле `department_id` таблицы `products`) и загрузим их в датафрейм Pandas. 

In [23]:
df_train = pd.read_sql_query("SELECT tr.order_id, tr.product_id, prod.product_name, tr.add_to_cart_order, tr.reordered "+
                             "FROM order_products__train as tr, orders as ord, products as prod "+
                             "WHERE ord.order_dow = '6' AND prod.department_id = '2' AND tr.order_id = ord.order_id "+
                             "AND tr.product_id = prod.product_id", conn)
df_train

,order_id,product_id,product_name,add_to_cart_order,reordered
0,1447810,26756,Light CocoWhip! Coconut Whipped Topping,18,1
1,112127,1892,Baby's Vitamin D3,10,0
2,2947672,38662,Roasted Almond Butter,7,0
3,1594674,32115,93/7 Ground Beef,8,1
4,19097,38662,Roasted Almond Butter,31,1
...,...,...,...,...,...
280,3064533,25802,Pierogi Potato & Cheese,3,1
281,2949070,20599,Unsalted Roasted Cashews,6,1
282,2832028,26756,Light CocoWhip! Coconut Whipped Topping,12,0
283,489262,43041,All Purpose Precision Tip 2 Pack,3,0


Количество строк:

In [58]:
len(df_train)

285

Количество товаров (столбец `product_id`):

In [60]:
prods = df_train.groupby(['product_id'])
prods.ngroups

145

### Задание 3
Определите пять наиболее популярных товаров в датафрейме транзакций и определите количество покупок (транзакций) этих товаров.

In [61]:
prods.size().nlargest(5)

product_id
38662    29
26756    19
20406    13
15551     8
25802     6
dtype: int64

### Задание 4
Постройте транзакционную базу данных из полученного датафрейма, используя в качестве идентификатора транзакции столбец `order_id`, а в качестве названий товаров - поле `product_name` из датафрейма для таблицы `products`, соответствующее столбцу `product_id`. Найдите в транзакционной базе данных три транзакции с наибольшим количеством товаров и выведите их на экран. 

In [88]:
df_t_database = df_train.groupby(['order_id'])['product_name'].apply(list).reset_index(name='product_name')
t_database = df_t_database.to_numpy()
t_database

array([['1002424', list(['Rye Flour'])],
       ['1008640', list(['SleepTabs Nighttime Sleep Aid'])],
       ['1009412',
        list(['Moderate Absorbency Long Length Incontinence Pads'])],
       ['10371', list(['Organic Garam Masala'])],
       ['1051349', list(['Cherry Vanilla Granola'])],
       ['1057798', list(['Light CocoWhip! Coconut Whipped Topping'])],
       ['1073899', list(["Original Blue Painter's Tape"])],
       ['1109561', list(['Deluxe Nut Mix'])],
       ['111603', list(['Roasted Unsalted Almonds'])],
       ['1116537', list(['100% Pure Eucalyptus'])],
       ['1117161', list(['Beef Chuck Roast'])],
       ['112127', list(["Baby's Vitamin D3"])],
       ['1133275', list(['Kiwifruit'])],
       ['1144246', list(['Polenta'])],
       ['1189708', list(['Recolte Wine'])],
       ['1206472', list(['Kiwifruit'])],
       ['1212831', list(['Gluten Free Fig Bars'])],
       ['1234933', list(['Black Elderberry Nighttime Syrup'])],
       ['123853', list(['Cherry Vanilla Gran

In [103]:
sorted_database = sorted(t_database, key=lambda database: len(database[1]), reverse = True)
sorted_database

[array(['1383780',
        list(['Arugula Rocket Salad', 'Parsley Italian Dark Green Flat'])],
       dtype=object), array(['1536712',
        list(['Healing Baby Diaper Rash Cream', 'Healing Ointment Advanced Therapy'])],
       dtype=object), array(['2104247',
        list(['Digital Pregnancy Test', 'Gold Digital Pregnancy Test'])],
       dtype=object), array(['2274109', list(['Aromatic Bitters', 'French Wine Aperitif'])],
       dtype=object), array(['2417734',
        list(['Organic Whole Wheat Couscous', 'Pitted Green Olives'])],
       dtype=object), array(['2631037',
        list(['Organic Tulsi Ginger Tea', 'Roasted Almond Butter'])],
       dtype=object), array(['2929827',
        list(['Psychic Detox Magickal Mineral Bath Salts', 'Protection Magickal Mineral Bath Salts With Natural Sea Salt & Pure Essential Oils'])],
       dtype=object), array(['3003270', list(['Organic Coco Whip', 'Cherry Vanilla Granola'])],
       dtype=object), array(['3216026',
        list(['Multivita

Транзакции с наибольшим количеством товаров:

In [104]:
for i in range(3):
    print(sorted_database[i])

['1383780'
 list(['Arugula Rocket Salad', 'Parsley Italian Dark Green Flat'])]
['1536712'
 list(['Healing Baby Diaper Rash Cream', 'Healing Ointment Advanced Therapy'])]
['2104247' list(['Digital Pregnancy Test', 'Gold Digital Pregnancy Test'])]


### Задание 5
Постройте по транзакционной базе данных бинарную базу данных в формате датафрейма пакета `mlxtend`. По бинарной базе данных определите пять  наиболее популярных товаров и определите количество покупок (транзакций) этих товаров.

Сохраним id заказов в отдельный список:

In [117]:
order_id_list = []
order_prod_list = []
for i in range(len(t_database)):
    order_id_list.append(t_database[i][0])
    order_prod_list.append(t_database[i][1])
order_prod_list

[['Rye Flour'],
 ['SleepTabs Nighttime Sleep Aid'],
 ['Moderate Absorbency Long Length Incontinence Pads'],
 ['Organic Garam Masala'],
 ['Cherry Vanilla Granola'],
 ['Light CocoWhip! Coconut Whipped Topping'],
 ["Original Blue Painter's Tape"],
 ['Deluxe Nut Mix'],
 ['Roasted Unsalted Almonds'],
 ['100% Pure Eucalyptus'],
 ['Beef Chuck Roast'],
 ["Baby's Vitamin D3"],
 ['Kiwifruit'],
 ['Polenta'],
 ['Recolte Wine'],
 ['Kiwifruit'],
 ['Gluten Free Fig Bars'],
 ['Black Elderberry Nighttime Syrup'],
 ['Cherry Vanilla Granola'],
 ['Pinot Noir Rose'],
 ['Crushed Chili'],
 ['Wooden Foot File'],
 ['Calendula with Beeswax & Sunflower Zinc Oxide Diaper Cream'],
 ['Boneless Pork Tenderloin'],
 ['Peppermint Essential Oil'],
 ['Creamer'],
 ["Baby's Jarro-Dophilus +FOS Powder Probiotic Supplement"],
 ['Light CocoWhip! Coconut Whipped Topping'],
 ['Deluxe Nut Mix'],
 ['Sunflower Seeds'],
 ['Melatonin 1 Mg Peppermint Sublingual Tablets'],
 ['Wood Clothespins'],
 ['Organic Garam Masala'],
 ['93/7 Grou

Построим бинарную базу данных в формате датафрейма пакета mlxtend.

In [119]:
from mlxtend.preprocessing import TransactionEncoder

te = TransactionEncoder()
dataset_bin = te.fit(order_prod_list).transform(order_prod_list)
binary_db = pd.DataFrame(dataset_bin, columns = te.columns_, index = order_id_list)
binary_db

,100% Pure Eucalyptus,5-HTP 100 Mg Vegetarian Capsules,93/7 Ground Beef,AA Rechargeable Nickel Metal Hydride Batteries,All Purpose Precision Tip 2 Pack,Aromatic Bitters,Arugula Rocket Salad,Baby Eczema Therapy Soothing Bath Treatment,Baby Healing Ointment,Baby Tummy Gripe Water Dietary Supplement,...,Unsalted Roasted Cashews,Vinho Verde Rose,Walnuts,Watch 2032BP Batteries,White Zinfandel,Whole Bay Leaves,Whole Cumin,Whole Raw Brazil Nuts,Wood Clothespins,Wooden Foot File
1002424,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1008640,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1009412,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
10371,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1051349,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
954716,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
95948,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
959540,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
961207,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Определим пять наиболее популярных товаров и количество покупок (транзакций) этих товаров по полученному датафрейму.

In [166]:
from mlxtend.frequent_patterns import apriori

itemsets_apr_2 = apriori(binary_db, min_support=6/binary_db.shape[0], use_colnames=True)
itemsets_apr_2

,support,itemsets
0,0.029197,(Kiwifruit)
1,0.069343,(Light CocoWhip! Coconut Whipped Topping)
2,0.021898,(Pierogi Potato & Cheese)
3,0.105839,(Roasted Almond Butter)
4,0.047445,(Roasted Unsalted Almonds)


In [171]:
for i in range(5):
    for j in itemsets_apr_2['itemsets'][i]: 
        print(binary_db[j].value_counts())

False    266
True       8
Name: Kiwifruit, dtype: int64
False    255
True      19
Name: Light CocoWhip! Coconut Whipped Topping, dtype: int64
False    268
True       6
Name: Pierogi Potato & Cheese, dtype: int64
False    245
True      29
Name: Roasted Almond Butter, dtype: int64
False    261
True      13
Name: Roasted Unsalted Almonds, dtype: int64


### Задание 6
При помощи метода построения популярных наборов предметов Apriori постройте популярный набор предметов с минимальной поддержкой не менее 3, имеющий максимальную длину. При отсутствии таких наборов уменьшите поддержку до 2. В случае нехватки вычислительных ресурсов (слишком долгой работы программы) при построении популярных наборов предметов сокращайте число записей в наборе данных (например, делая выборку половины записей набора).

При поддержке 3 и 2 все наборы состоят только из 1 элемента.

In [136]:
from mlxtend.frequent_patterns import apriori

itemsets_apr_ = apriori(binary_db, min_support=3/binary_db.shape[0], use_colnames=True)
itemsets_apr_

,support,itemsets
0,0.018248,(93/7 Ground Beef)
1,0.010949,(All Purpose Precision Tip 2 Pack)
2,0.010949,(Black Chia Seeds)
3,0.010949,(Cherry Vanilla Granola)
4,0.010949,(Classic Vanilla Coffee Creamer)
5,0.014599,(Coconut Almond Granola)
6,0.018248,(Creamer)
7,0.010949,(Deluxe Nut Mix)
8,0.010949,(Early Result Pregnancy Test)
9,0.010949,(Falafel)


Для построения набора ассоциативных правил добавим в подборку популярных наборов товаров наборы с более низкой поддержкой, но у которых количество продуктов > 1.

In [137]:
itemsets_apr = apriori(binary_db, min_support=1/binary_db.shape[0], use_colnames=True)
itemsets_apr

,support,itemsets
0,0.003650,(100% Pure Eucalyptus)
1,0.003650,(5-HTP 100 Mg Vegetarian Capsules)
2,0.018248,(93/7 Ground Beef)
3,0.003650,(AA Rechargeable Nickel Metal Hydride Batteries)
4,0.010949,(All Purpose Precision Tip 2 Pack)
...,...,...
151,0.003650,"(Organic Tulsi Ginger Tea, Roasted Almond Butter)"
152,0.003650,"(Organic Whole Wheat Couscous, Pitted Green Ol..."
153,0.003650,"(Vinho Verde Rose, Pinot Noir Rose)"
154,0.003650,(Protection Magickal Mineral Bath Salts With N...


### Задание 7
Используя пакет `mlxtend` или реализацию на Python, постройте набор ассоциативных правил для полученного популярного наборов товаров . Используйте уровень достоверности (confidence), равный 0.6.

In [155]:
from mlxtend.frequent_patterns import association_rules

association_df = association_rules(itemsets_apr, metric="confidence", min_threshold=0.6)
association_df

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(French Wine Aperitif),(Aromatic Bitters),0.00365,0.003650,0.00365,1.0,274.000000,0.003636,inf
1,(Aromatic Bitters),(French Wine Aperitif),0.00365,0.003650,0.00365,1.0,274.000000,0.003636,inf
2,(Parsley Italian Dark Green Flat),(Arugula Rocket Salad),0.00365,0.003650,0.00365,1.0,274.000000,0.003636,inf
3,(Arugula Rocket Salad),(Parsley Italian Dark Green Flat),0.00365,0.003650,0.00365,1.0,274.000000,0.003636,inf
4,(Digital Pregnancy Test),(Gold Digital Pregnancy Test),0.00365,0.003650,0.00365,1.0,274.000000,0.003636,inf
5,(Gold Digital Pregnancy Test),(Digital Pregnancy Test),0.00365,0.003650,0.00365,1.0,274.000000,0.003636,inf
6,"(Multivitamin, Kids Complete, Gummies)",(Facial Mask Age Defying Hydro Serum),0.00365,0.003650,0.00365,1.0,274.000000,0.003636,inf
7,(Facial Mask Age Defying Hydro Serum),"(Multivitamin, Kids Complete, Gummies)",0.00365,0.003650,0.00365,1.0,274.000000,0.003636,inf
8,(Healing Baby Diaper Rash Cream),(Healing Ointment Advanced Therapy),0.00365,0.007299,0.00365,1.0,137.000000,0.003623,inf
9,(Organic Tulsi Ginger Tea),(Roasted Almond Butter),0.00365,0.105839,0.00365,1.0,9.448276,0.003263,inf


### Задание 8
Для построенного набора ассоциативных правил вычислите показатель (меру) оценки ассоциативных правил, указанную в индивидуальном задании, и определите ассоциативные правила с наилучшим значением показателя оценки.


In [161]:
association_df["lift"].sort_values(ascending=False)

11    274.000000
10    274.000000
7     274.000000
6     274.000000
5     274.000000
4     274.000000
3     274.000000
2     274.000000
1     274.000000
0     274.000000
13    137.000000
8     137.000000
14     54.800000
12     54.800000
9       9.448276
Name: lift, dtype: float64